In [6]:
pip install -U "huggingface_hub[cli]"

   ---------------------------------------- 0.0/67.7 kB ? eta -:--:--
   ------------------------------------ --- 61.4/67.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 67.7/67.7 kB 736.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/447.5 kB ? eta -:--:--
   -------------------- ------------------- 225.3/447.5 kB 4.6 MB/s eta 0:00:01
   -------------------- ------------------- 225.3/447.5 kB 4.6 MB/s eta 0:00:01
   ------------------------ --------------- 276.5/447.5 kB 1.7 MB/s eta 0:00:01
   ------------------------------ --------- 337.9/447.5 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------  440.3/447.5 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 447.5/447.5 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.4
    Uninstalling huggingface-hub-0.23.4:
      Successfully uninstalled huggingface-hub-0.23.4
Note: you may need to restart 


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install transformers datasets accelerate


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load CSV
data = pd.read_csv("data\Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")

# Filter data jika diperlukan (contoh: hanya gunakan data tertentu berdasarkan flags)
# data = data[data['flags'] == 'use']

# Gabungkan instruction, intent, dan lainnya menjadi prompt
def create_prompt(row):
    instruction = row['instruction']
    intent = row['intent']
    category = row['category']
    # Gabungkan menjadi format prompt yang sesuai
    return f"Instruction: {instruction}\nIntent: {intent}\nCategory: {category}\n"

data['prompt'] = data.apply(create_prompt, axis=1)

# Split data menjadi train dan test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Simpan sebagai JSONL
train_data[['prompt', 'response']].to_json("data/train_data.jsonl", orient="records", lines=True)
test_data[['prompt', 'response']].to_json("data/test_data.jsonl", orient="records", lines=True)


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

from transformers import LlamaForCausalLM, LlamaTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

# Load model and tokenizer
model_name = "model/llama-model"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

# Load dataset
dataset = load_dataset("json", data_files={
    "train": "data/train_data.jsonl",
    "validation": "data/test_data.jsonl"
})

# Tokenize data
def preprocess_function(examples):
    inputs = examples["prompt"]
    outputs = examples["response"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(outputs, max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    fp16=True  # Enable if using GPU
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Fine-tune model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("model/fine_tuned_llama")
tokenizer.save_pretrained("model/fine_tuned_llama")


NameError: name '_C' is not defined

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

# Generate a response
def chat(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
print(chat("Apa itu AI?"))


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B.
401 Client Error. (Request ID: Root=1-6741e028-53c81a4f6c6d58b17a1f580c;a40da153-3d7b-4388-a154-45b9004eb6f6)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
brew install huggingface-cli

In [2]:
from transformers import LlamaModel, LlamaConfig

# Initializing a LLaMA llama-7b style configuration
configuration = LlamaConfig()

# Initializing a model from the llama-7b style configuration
model = LlamaModel(configuration)

# Accessing the model configuration
configuration = model.config



: 

In [1]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")
pipeline("Hey how are you doing today?")

c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
401 Client Error. (Request ID: Root=1-6741e10e-3d1ab6c4307297926c4ef1dc;c28d256f-b97a-43ae-9cc9-48b7a61c6136)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must have access to it and be authenticated to access it. Please log in.